In [ ]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn
import torchvision
from torchvision import datasets
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
device = torch.device('cpu')

In [ ]:
class MNISTSampler:
    def __init__(self, train=True):
        dataset = datasets.MNIST(
            './mnist', train=train, download=True,
            transform=torchvision.transforms.ToTensor())
        self.loader = DataLoader(dataset, batch_size=128, shuffle=True)
    def sample(self):
        while True:
            for batch in self.loader:
                yield batch

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_layers=1, num_units=20, sampler=None, **kwargs):
        super().__init__()
        self.num_layers = num_layers
        if kwargs:
            self.params = kwargs
        else:
            self.params = {}
            in_size = 784
            for i in range(num_layers):
                self.params['linear_{:d}_W'.format(i)] =\
                    nn.Parameter(torch.randn(in_size, num_units) * 0.001)
                self.params['linear_{:d}_b'.format(i)] =\
                    nn.Parameter(torch.zeros(1, num_units))
                in_size = num_units
            self.params['final_W'] =\
                nn.Parameter(torch.randn(in_size, 10) * 0.001)
            self.params['final_b'] =\
                nn.Parameter(torch.zeros(1, 10))
            self.mods = nn.ParameterList([v for v in self.params.values()])
        self.act = nn.Sigmoid()
        self.loss = nn.CrossEntropyLoss()
        self.sampler = sampler
        if not sampler:
            self.sampler = MNISTSampler().sample()
    def _linear_forward(self, h, W, b, act=True):
        h = torch.matmul(h, W) + b
        if act:
            h = self.act(h)
        return h
    def forward(self):
        batch = next(self.sampler)
        x, target = batch
        h = x.view(x.size(0), -1)
        for i in range(self.num_layers):
            h = self._linear_forward(h, self.params['linear_{:d}_W'.format(i)], self.params['linear_{:d}_b'.format(i)])
        y = self._linear_forward(h, self.params['final_W'], self.params['final_b'], act=False)
        return self.loss(y, target)

In [ ]:
class SGD:
    def __init__(self, lr=1e-2):
        self.lr = lr
    def __call__(self, grads):
        updates = {}
        for k, v in grads.items():
            updates[k] = -self.lr * v
        return updates
class RMSProp:
    def __init__(self, lr=1e-2, decay=0.99):
        self.lr = lr
        self.decay = decay
        self.state = None
    def __call__(self, grads):
        if self.state is None:
            self.state = {}
            for k, v in grads.items():
                self.state[k] = torch.zeros_like(v)
        updates = {}
        for k, v in grads.items():
            state = self.state[k]
            state = self.decay * state + (1 - self.decay) * torch.pow(v, 2)
            self.state[k] = state
            updates[k] = -self.lr * v / (torch.sqrt(state) + 1e-6)
        return updates

In [ ]:
losses = {'iter': [], 'loss': [], 'optim': []}

In [ ]:
for _ in range(10):
    mlp = MLP()
    sgd = torch.optim.RMSprop(mlp.parameters(), lr=1e-3)
    for i in range(100):
        y = mlp()
        losses['iter'].append(i)
        losses['loss'].append(float(y.data.cpu().numpy()))
        losses['optim'].append('SGD')
        mlp.zero_grad()
        y.backward()
        sgd.step()
    print(y.item())

In [ ]:
for _ in range(10):
    mlp = MLP()
    sgd = SGD()
    for i in range(100):
        y = mlp()
        losses['iter'].append(i)
        losses['loss'].append(float(y.data.cpu().numpy()))
        losses['optim'].append('SGD')
        mlp.zero_grad()
        y.backward()
        grads = {}
        for k in mlp.params.keys():
            grads[k] = mlp.params[k].grad.detach()
        updates = sgd(grads)
        for k in mlp.params.keys():
            mlp.params[k].data += updates[k]
    print(y.item())

In [ ]:
for _ in range(10):
    mlp = MLP()
    rms = RMSProp()
    for i in range(100):
        y = mlp()
        losses['iter'].append(i)
        losses['loss'].append(float(y.data.cpu().numpy()))
        losses['optim'].append('RMSProp')
        mlp.zero_grad()
        y.backward()
        grads = {}
        for k in mlp.params.keys():
            grads[k] = mlp.params[k].grad.detach()
        updates = rms(grads)
        for k in mlp.params.keys():
            mlp.params[k].data += updates[k]
    print(y.item())

In [ ]:
seaborn.lineplot(x='iter', y='loss', hue='optim', data=losses)
plt.show()

In [ ]:
class MetaOptimizer(nn.Module):
    def __init__(self, dim=1, hidden=20, out=1, layers=2):
        super().__init__()
        self.hidden = hidden
        self.layers = layers
        self.lstm = nn.LSTM(input_size=dim, hidden_size=hidden, num_layers=layers)
        self.linear = nn.Linear(hidden, out)
    def forward(self, grad, state=None):
        # seq_len x batch x size
        grad = grad.view(1, -1, 1)
        if state is None:
            h = torch.zeros(self.layers, grad.size()[1], self.hidden)
            c = torch.zeros(self.layers, grad.size()[1], self.hidden)
            h = h.to(device)
            c = c.to(device)
            state = (h, c)
        lstm_out, state = self.lstm(grad, state)
        # lstm_out.shape: seq_len x batch x hidden
        update = self.linear(lstm_out.view(-1, self.hidden))
        return update, state

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
meta_optimizer = MetaOptimizer()
meta_optimizer_optim = torch.optim.Adam(meta_optimizer.parameters(), lr = 1e-4)
meta_optimizer = meta_optimizer.to(device)

In [ ]:
out_mul = 0.1
pbar = tqdm(range(500))
state = None
for k in pbar:
    mlp = MLP()
    states = {k: None for k in mlp.params.keys()}
    for l in range(5):
        loss = 0.
        temp_params = {}
        for i in range(20):
            y = mlp()
            loss += y
            mlp.zero_grad()
            y.backward(retain_graph=True)
            grads = {}
            for k in mlp.params.keys():
                grads[k] = mlp.params[k].grad.detach()
            updates = {}
            for k in grads.keys():
                updates[k], states[k] = meta_optimizer(grads[k], states[k])
            for k in updates.keys():
                temp_params[k] = mlp.params[k] + updates[k].view_as(mlp.params[k]) * out_mul
                temp_params[k].retain_grad()
            mlp = MLP(**temp_params)
        for k in states.keys():
            states[k] = (states[k][0].detach().clone(), states[k][1].detach().clone())
        meta_optimizer_optim.zero_grad()
        loss.backward(retain_graph=True)
        meta_optimizer_optim.step()
    pbar.set_description('Loss: %.3f' % loss.item())

In [ ]:
meta_optimizer.eval()
for _ in range(10):
    q = QuadraticModel()
    state = None
    for i in range(100):
        y = q()
        losses['iter'].append(i)
        losses['loss'].append(float(y.data.cpu().numpy()))
        losses['optim'].append('L2L')
        grads = torch.autograd.grad(y, q.theta)[0].detach()
        update, state = meta_optimizer(grads, state)
        update = update.view_as(q.theta.data)
        q.theta.data += update

In [ ]:
ax = seaborn.lineplot(x='iter', y='loss', hue='optim', data=losses)
ax.set_yscale('log')
plt.show()

In [ ]:
torch.save(meta_optimizer.state_dict(), 'meta-qf.pt')

In [ ]:
df[df.optim == 'L2L'].groupby('iter').mean()

In [ ]:
df[df.optim == 'RMSProp'].groupby('iter').mean()